## Question 1
implement a FA function with parameters: the dataset and the desired numbers of factors. These part is a bit like the PCA function implemented in the last homework.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
def manual_fa(X, num_factors):
    # Compute the covariance matrix
    cov_mx = np.cov(X.T)

    # Compute the eigenvectors and eigenvalues
    eigvals, eigvecs = np.linalg.eig(cov_mx)

    # Sort the eigenvectors and eigenvalues in descending order
    sort_idx = np.argsort(eigvals)[::-1]
    eigvals = np.real(eigvals[sort_idx])
    eigvecs = np.real(eigvecs[:, sort_idx])


    # Compute the loading matrix A
    loading_matrix = eigvecs[:, :num_factors]
    print(loading_matrix.shape)

    # Compute the factor matrix F
    factor_matrix = np.dot(X,loading_matrix)

    # Compute the communality factor of (h_i)^2
    communality = np.sum(loading_matrix**2, axis=1)
    
    # Compute the uniqueness vector of ψ_i
    uniqueness = 1 - communality

    # Compute the vector of the proportions of total variance contributed by the ith factor
    prop_var = eigvals[:num_factors] / np.sum(eigvals)

    return loading_matrix, factor_matrix, communality, uniqueness, prop_var

In [7]:
# Just a test case.

from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
loading_matrix, factor_matrix, communality, uniqueness, prop_va = manual_fa(X,2)
# print("Loading Matrix:\n", loading_matrix)
# print("\nfactor_matrix:\n", factor_matrix)
# print("\ncommunality:\n", communality)
# print("\nuniqueness:\n", uniqueness)
# print("\nprop_va:\n", prop_va)


(4, 2)


### question b
On the AutoMPG dataset, compare with the PCA results in HW05.

In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/autompg.csv')
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)
autompg_variables = df.drop(['mpg','car name'], axis=1)
autompg_mpg = df['mpg']
autompg_variables = pd.DataFrame(autompg_variables, dtype=float)

loading_matrix, factor_matrix, communality, uniqueness, prop_va = manual_fa(autompg_variables, 2)
# print("Loading Matrix:\n", loading_matrix)
# print("\nfactor_matrix:\n", factor_matrix)
# print("\ncommunality:\n", communality)
# print("\nuniqueness:\n", uniqueness)
# print("\nprop_va:\n", prop_va)

(7,)


We can see that the structure of two model are different. The FA use X=FA, and the PCA use Z=XV.

## Question 2: 
Transpose the ORL face dataset to be a data matrix of 2576x400. Perform the factore matrix. We take n=400 and see how many variables are needed to get the desired explanation.

In [9]:
from PIL import Image

def edge(datas):
    criteria = []
    element = 0
    for i, nums in enumerate(datas):
        element += nums
        if element >= 0.5 and len(criteria) == 0:
            criteria.append(i+1)
        if element >= 0.6 and len(criteria) == 1:
            criteria.append(i+1)
        if element >= 0.7 and len(criteria) == 2:
            criteria.append(i+1)
        if element >= 0.8 and len(criteria) == 3:
            criteria.append(i+1)
        if element >= 0.9 and len(criteria) == 4:
            criteria.append(i+1)
            break
    return criteria

ORLface_data = []
for i in range(1, 41):
    for j in range(1,11):
        image_dir = f"/content/drive/MyDrive/DA/ORL Faces/{i}_{j}.png"
        img = Image.open(image_dir)
        img_array = np.asarray(img)
        ORLface_data.append(img_array.flatten())
ORLface_data = np.array(ORLface_data)

loading_matrix, factor_matrix, communality, uniqueness, prop_va = manual_fa(ORLface_data, 400)
print("Loading Matrix:\n", loading_matrix)
print("\nfactor_matrix:\n", factor_matrix)
print("\ncommunality:\n", communality)
print("\nuniqueness:\n", uniqueness)
print("\nprop_va:\n", prop_va)

criteria = edge(prop_va)
print("50% explanable:", criteria[0], " principal components")
print("60% explanable:", criteria[1], " principal components")
print("70% explanable:", criteria[2], " principal components")
print("80% explanable:", criteria[3], " principal components")
print("90% explanable:", criteria[4], " principal components")

(2576, 400)
Loading Matrix:
 [[ 4.18418191e-03  2.92082392e-02  3.98717037e-02 ...  1.08463577e-02
  -3.88942168e-03 -2.43641364e-03]
 [ 4.34579327e-03  2.92119864e-02  3.97740019e-02 ...  9.40164343e-03
  -1.01740124e-03 -2.45015920e-03]
 [ 3.69863744e-03  2.93623995e-02  4.00714546e-02 ...  1.27420187e-02
   4.05565911e-03  5.34254516e-03]
 ...
 [ 1.53271003e-02 -1.27914953e-02  2.41267237e-02 ... -3.11265398e-02
  -2.33792628e-02 -2.19128626e-06]
 [ 1.33515940e-02 -1.78069997e-02  2.89114944e-02 ... -1.43982079e-03
  -4.05094260e-03  2.61679055e-03]
 [ 1.38193377e-02 -1.75111834e-02  3.08289732e-02 ... -2.35419739e-03
  -4.65625113e-02 -7.79406816e-03]]

factor_matrix:
 [[-3735.25330174  3663.54695712   761.4230817  ...   -15.44696504
     32.79484107    10.95171292]
 [-4020.91757202  3539.56387202   398.21358833 ...   -14.493715
     28.63463334    10.95171292]
 [-4201.5410653   2920.14627233   298.77109127 ...    -8.79447361
     31.76165768    10.95171292]
 ...
 [-4339.84011576  

* 50% explanable: 5  principal components
* 60% explanable: 9  principal components
* 70% explanable: 16  principal components
* 80% explanable: 32  principal components
* 90% explanable: 76  principal components
* If we want to plot an image with 80% explaining of the total variance, we take n=32.

In [8]:
loading_matrix, factor_matrix, communality, uniqueness, prop_va = manual_fa(ORLface_data, 32)
print(loading_matrix.shape)

# first_PC = loading_matrix[:, 0]

# min_value = np.min(first_PC)
# max_value = np.max(first_PC)
# scaled_pc = (first_PC - min_value) * (255 / (max_value - min_value))

# scaled_pc_2D = scaled_pc.reshape(56,46)
# img = Image.fromarray(scaled_pc_2D.astype('uint8'))
# img.save('r.png', 'PNG')
# img.show()

NameError: ignored

## Question 3
### a
find a oackage to perform PLSR on the AutoMPG dataset. Take 300 cars randomly to build the model and the rest 92 cars to test.
In PLSR, we can see the relationship of multiple x and multiple y altogether, and see if the relashionship are strong and give us high R^2 score.

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/drive/MyDrive/autompg.csv')
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)
data = pd.DataFrame(df)

# Split data into training and testing sets
train, test = train_test_split(data, test_size=92)
X_train = train.drop(['mpg', 'car name'], axis=1)
y_train = train['mpg']
X_test = test.drop(['mpg', 'car name'], axis=1)
y_test = test['mpg']

# Fit PLSR model
plsr = PLSRegression(n_components=2)
plsr.fit(X_train, y_train)

# Predict on test set
y_pred = plsr.predict(X_test)
# print(y_pred)
# Print R^2 score
score = plsr.score(X_test, y_test)
print(f'R^2 score: {score:.3f}')


R^2 score: 0.792


We can see that the R^2 is 0.792, which is really a nice number! However, if we take the model year as a part of y, things will be different.

In [ ]:
# Split data into training and testing sets
train, test = train_test_split(data, test_size=92, random_state=42)
X_train = train.drop(['mpg', 'car name', 'model year'], axis=1)
y_train = train[['mpg', 'model year']]
X_test = test.drop(['mpg', 'car name', 'model year'], axis=1)
y_test = test[['mpg', 'model year']]

# Fit PLSR model
plsr = PLSRegression(n_components=2)
plsr.fit(X_train, y_train)

# Predict on test set
y_pred = plsr.predict(X_test)
# print(y_pred)
# Print R^2 score
score = plsr.score(X_test, y_test)
print(f'R^2 score: {score:.3f}')


R^2 score: 0.355


We can see the R^2 score drop dramatically. In my opinion, the model year shouldn't be in the dependent part, since it makes the result worse.